<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/dev-branch/featuretools_learning_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install tornado==4.5.3 featuretools==0.6.1 pandas==0.24.1

In [0]:
import numpy as np
import pandas as pd
import featuretools as ft
from featuretools.primitives import make_trans_primitive
import featuretools.variable_types as vt

In [0]:
# Go to the CSV file in Google Drive, right-click on it and select 
# “Get shareable link”. The link will be copied into your clipboard. 
# Paste this link into following a string variable i.e. link:

link = 'https://drive.google.com/open?id=10JNCo2Opquuq2cwGiyGgvlds8U2wz3mW'  # The shareable link

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


fluff, id = link.split('=')   # Verify that you have everything after '=' by using print(id)

# Finally reading the data into a dataframe

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dummy_file_name.csv')  
df1 = pd.read_csv('dummy_file_name.csv')

In [0]:
df1 = df1.iloc[:,1:df1.shape[1]]   # Target Variable must be removed for feature augmentation
df1['df_index']= np.arange(df1.shape[0])

#df1 = pd.DataFrame({'df_index' : [1,2,3,4,5],
#                 'location':['aust','aust','aust','canada','canada'],
#                  'prices':[34,52,46,25,67],
#                   'values':[786,345,123,654,841],
#                    'cat':[1,1,2,1,2]
#                  })


In [18]:
df1.head()

,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,...,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker,df_index
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,1,1,1,0
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,2,1,1,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,1,1,1,2
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,2,1,2,3
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,1,1,2,4


In [0]:
def log10(column):
  return np.log10(column)

Log10 = make_trans_primitive(function=log10,
                             input_types=[vt.Numeric],
                             return_type=vt.Numeric)

In [19]:
#df1['date']=pd.date_range(start='02/07/2019',periods=5, freq='D')

es = ft.EntitySet(id='Transactions')
es.entity_from_dataframe(entity_id='log', 
                         dataframe=df1,
                         index='df_index'
                        )

es.normalize_entity(base_entity_id='log', new_entity_id='loc', 
                   index= 'Purpose' )

Entityset: Transactions
  Entities:
    log [Rows: 1000, Columns: 21]
    loc [Rows: 10, Columns: 1]
  Relationships:
    log.Purpose -> loc.Purpose

In [20]:
fm, features = ft.dfs(entityset=es, target_entity='log',
                      verbose = True                        
                     )
#max_depth = 2, n_jobs = 1 ,trans_primitives = ['add', 'multiply', Log10], agg_primitives = ['sum', 'mean'],

Built 135 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


In [22]:
fm

,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,...,loc.MEAN(log.Most valuable available asset),loc.MEAN(log.Age (years)),loc.MEAN(log.Concurrent Credits),loc.MEAN(log.Type of apartment),loc.MEAN(log.No of Credits at this Bank),loc.MEAN(log.Occupation),loc.MEAN(log.No of dependents),loc.MEAN(log.Telephone),loc.MEAN(log.Foreign Worker),loc.COUNT(log)
df_index,,,,,,,,,,,,,,,,,,,,,
0,1,18,4,2,1049,1,2,4,2,1,...,2.254144,32.403315,2.696133,1.795580,1.320442,2.922652,1.088398,1.348066,1.038674,181
1,1,9,4,0,2799,1,3,2,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
2,2,12,2,9,841,2,4,2,2,1,...,2.278351,35.350515,2.484536,1.886598,1.556701,2.907216,1.154639,1.536082,1.000000,97
3,1,12,4,0,2122,1,3,3,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
4,1,12,4,0,2171,1,3,4,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
5,1,10,4,0,2241,1,2,1,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
6,1,8,4,0,3398,1,4,1,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
7,1,6,4,0,1361,1,2,2,3,1,...,2.311966,37.089744,2.670940,1.952991,1.444444,2.799145,1.222222,1.371795,1.089744,234
8,4,18,4,3,1098,1,1,4,2,1,...,2.182143,34.767857,2.735714,1.917857,1.378571,2.875000,1.107143,1.339286,1.017857,280
